In [2]:
import torch 
import math

In [7]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3]) # Polynomial order
xx = x.unsqueeze(-1).pow(p) # X vals before weights

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten

tensor([[ -3.1416,   9.8696, -31.0063],
        [ -3.1384,   9.8499, -30.9133],
        [ -3.1353,   9.8301, -30.8205],
        ...,
        [  3.1353,   9.8301,  30.8205],
        [  3.1384,   9.8499,  30.9133],
        [  3.1416,   9.8696,  31.0063]])
